# 1. Setting

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 904 kB 5.0 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe 

In [ ]:
# 필요한 라이브러리 세팅
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta

!pip install tqdm
from tqdm import tqdm

import pandas as pd
import numpy as np

In [ ]:
## ipynb 파일 import
# !pip install import_ipynb # import_ipynb 모듈 설치
# import import_ipynb

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

# url='https://www.yanolja.com/hotel'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


# 2. Crawling Review

### 2-1. 특수문자 제거, 띄어쓰기 제거, 20글자 이상 리뷰 불러오기 함수 정의

In [ ]:
import re

# 특수문자 제거 함수 정의
def remove_special_character(text):
  hangul = re.compile(r'[^ ㄱ-ㅣ 가-힣]') # 정규표현식에서 특수문자 제거를 위한 유니코드 적용
  hangul = hangul.sub('', text) # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
  return hangul

In [ ]:
# # 띄어쓰기 제거 함수 정의
# def apply_regular_expression_2(sentence):
#   pattern = re.compile(r'\s+') # 정규표현식에서 띄어쓰기 제거를 위한 유니코드 적용
#   result = pattern.sub('', sentence) #위에 설정한 "pattern" 규칙을 "sentence"에 적용시킴
#   return result

In [ ]:
# # 20글자 이상 리뷰 모음
# # over_20이라는 새로운 리스트 생성
# over_20 = []
# # for loop이용: 특수문자와 띄어쓰기 모두 제거한 arr이라는 리스트에서 글자수 20이상인 리뷰만 추출 -> "over_20"이라는 빈 리스트에 추가
# for i in range(len(arr)):
#   if len(arr[i]) >= 20:
#     over_20.append(arr[i])
# # result: over_20리스트 안에 20글자 이상인 것만 모두 저장;
# print(over_20)

### 2-2. 리뷰 크롤링 함수 정의

In [ ]:
def change_date(x):
    now = datetime.datetime.today()
    today = now.strftime("%Y. %m. %d")
    yesterday = (now - timedelta(days=1)).strftime("%Y. %m. %d")
    _2days_ago = (now - timedelta(days = 2)).strftime("%Y. %m. %d")
    _3days_ago = (now - timedelta(days = 3)).strftime("%Y. %m. %d")
    hour = int(datetime.datetime.today().strftime("%H"))
    if "시간 전" in x:
        x = int(x.replace("시간 전", ""))
        if hour - x > 0: return today
        else: return yesterday
    elif "일 전" in x:
        if "1" in x: return yesterday
        elif "2" in x: return _2days_ago
        else: return _3days_ago
    else: return x

In [ ]:
def yanolja_crawl(url):
  global review_df # 전역 변수를 지역 범위에서 적용
  driver.get(url)

  hotel = driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf h1').text
  hotel = hotel.replace("[★숙박대전] ", "")

  # 리뷰 페이지 클릭 (Click review page)
  driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf > a').click()

  # 무한 스크롤
  time.sleep(1)

  SCROLL_PAUSE_TIME = 1.5

  # Get scroll height : 스크롤 높이 가져오기
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
      # Scroll down to bottom : 스크롤 끝까지 다운
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

      # Wait to load page : 1.5초 대기
      time.sleep(SCROLL_PAUSE_TIME)
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
      time.sleep(SCROLL_PAUSE_TIME)

      # Calculate new scroll height and compare with last scroll height : 스크롤 다운 후 스크롤 높이 다시 가져오기
      new_height = driver.execute_script("return document.body.scrollHeight")

      if new_height == last_height:
        break
      
      last_height = new_height

  # 스크롤 끝나고 페이지 elements 가져오기
  raw = driver.page_source # 페이지의 elements 모두 가져오기
  html = BeautifulSoup(raw, "html.parser")

  # 리뷰 텍스트 선택자: div.item-wrap
  reviews = html.select("div.item-wrap")

  columns = ['hotel', 'star', 'review', 'clean_review', 'date']

  for r in reviews:
      review = r.select_one("p")
      date = r.select_one('time')
      date = change_date(date.text)
      year = float(date.split('.')[0]) # date.text : 2020. 01. 01 형태
      month = float(date.split('.')[1])

      stars = r.select('.container.score')
      for s in stars:
          star_count = 5 - len(s.select(".ico.gray"))
      
      # 특수문자 제거하고, 20글자 이상인 리뷰들만 df 생성
      review20 = remove_special_character(review.text)
      
      if len(review20) >= 15 and len(review20) <= 500 and year >= 2019 and month >= 6 : # 리뷰가 15자 이상 500자 이하, 날짜가 2019.06. 이후
        review_df = review_df.append(pd.DataFrame([[hotel, star_count, review.text, review20, date]], columns = columns),
                      ignore_index=True)

# 3. 제주 호텔 리스트 불러와서 함수 적용

In [ ]:
hotel_list_url = '/content/drive/MyDrive/Codepresso/TeamProject/data/jeju_hotel_list.csv'
hotel_list = pd.read_csv(hotel_list_url)
hotel_list = hotel_list[['hotel_code', 'hotel_name']]
hotel_list

,hotel_code,hotel_name
0,3009313,아트스테이 서귀포 하버


In [ ]:
# url_list에 호텔 코드 모두 넣어서 리스트 생성
url_list = []

for i in range(0, len(hotel_list)):
  url_list.append('https://www.yanolja.com/hotel/'+ np.str(hotel_list['hotel_code'][i]))
url_list[:3]

['https://www.yanolja.com/hotel/3009313',
 'https://www.yanolja.com/hotel/3008436',
 'https://www.yanolja.com/hotel/3010282']

In [ ]:
# yanolja_crawl 함수 불러오기 전에 review_df(DataFrame) 정의
columns = ['hotel', 'star', 'review', 'clean_review', 'date']
review_df = pd.DataFrame(columns = columns) # 데이터프레임 생성

for url in tqdm(url_list[:41]): #0~40
  yanolja_crawl(url)

review_df.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/jeju_hotel_review_raw.csv')

driver.close()

In [ ]:
review_url = '/content/drive/MyDrive/Codepresso/TeamProject/data/jeju_hotel_review_raw.csv'
hotel_review_df = pd.read_csv(review_url)

In [ ]:
hotel_review_df = hotel_review_df.drop([hotel_review_df.columns[0]], axis='columns')
hotel_review_df.tail()

,hotel,star,review,clean_review,date
12915,플레이스 캠프 제주,5,대중교통 이용이 편하고 좋았어요,대중교통 이용이 편하고 좋았어요,2019. 06. 29
12916,플레이스 캠프 제주,5,혼자 가는 여행이라면 무조건 강추!!!!,혼자 가는 여행이라면 무조건 강추,2019. 06. 22
12917,플레이스 캠프 제주,4,생각했던 것 보다 너무 좋았어요 시설도 너무 만족합니다^^\n다음 여행때는 꼭 연박...,생각했던 것 보다 너무 좋았어요 시설도 너무 만족합니다다음 여행때는 꼭 연박으로 하려구요,2019. 06. 18
12918,플레이스 캠프 제주,5,굉장히 좋았습니다\n크기는 더블룸은 혼자 쓰기에 괜찮고 둘이쓰기엔 좁아보입니다 침구...,굉장히 좋았습니다크기는 더블룸은 혼자 쓰기에 괜찮고 둘이쓰기엔 좁아보입니다 침구는 ...,2019. 06. 16
12919,플레이스 캠프 제주,1,세상세상 후기를 보고 숙박을해야한다고 느낀곳임,세상세상 후기를 보고 숙박을해야한다고 느낀곳임,2019. 06. 14


In [ ]:
hotel_review_df.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/jeju_hotel_review_raw.csv')